# Visual Oddball Stimulus Trail Type Classification #

We use tensor-based Discriminant analysis from Li, Q., & Schonfeld, D. (2014). Multilinear discriminant analysis for higher-order tensor data classification. IEEE transactions on pattern analysis and machine intelligence, 36(12), 2524-2537. to conduct a classification analysis. The data is the EEG part from a multimodal neuroimaging experiment. This task is a comparison to our multimodal Coupled Tensor Classification 

In [1]:
import numpy as np 
from hdf5storage import loadmat
import MDA 
from classification_tools import classification_metrics
from sklearn.model_selection import StratifiedKFold

df = loadmat('EEG_Visual_df.mat')['Xdata']
y = loadmat('EEG_Visual_label.mat')['ylabel']
y = y - 1   # convert labels to 0-1

# Convert data to list as it is required by our function application
Xdata = [df[:, :, :, j] for j in range(90)]
y = y.reshape(90).tolist()

In [2]:
print(len(Xdata), len(y))

90 90


Perfrom stratified 9 folds corss validation to evaluate model performance. The percentages of samples for each class are the same in each fold.

In [3]:
from tensorly.tenalg import multi_mode_dot

In [6]:
np.random.seed(999)
acc_CMDA, prec_CMDA, rec_CMDA, spec_CMDA = [], [], [], []
acc_DGTDA, prec_DGTDA, rec_DGTDA, spec_DGTDA = [], [], [], []
skf = StratifiedKFold(n_splits= 9)
for train_indx, test_indx in skf.split(np.zeros(len(Xdata)), y):
    Xtrain = [Xdata[i] for i in train_indx]
    Xtest = [Xdata[j] for j in test_indx]
    ytrain = [y[i] for i in train_indx]
    ytest = [y[j] for j in test_indx]
    U_CMDA = MDA.CMDA(Xtrain, ytrain)
    U_DGTDA = MDA.DGTDA(Xtrain, ytrain)
    Xtrain_CMDA = [multi_mode_dot(x, U_CMDA) for x in Xtrain]
    Xtrain_DGTDA = [multi_mode_dot(x, U_DGTDA) for x in Xtrain]
    Xtest_CMDA = [multi_mode_dot(z, U_CMDA) for z in Xtest]
    Xtest_DGTDA = [multi_mode_dot(z, U_DGTDA) for z in Xtest]
    y_CMDA = MDA.Tensor_NN(Xtrain_CMDA, ytrain, Xtest_CMDA, 1)
    y_DGTDA = MDA.Tensor_NN(Xtrain_DGTDA, ytrain, Xtest_DGTDA, 1)
    a, b, c, d = classification_metrics(ytest, y_CMDA)
    acc_CMDA.append(a)
    prec_CMDA.append(b)
    rec_CMDA.append(c)
    spec_CMDA.append(d)
    a, b, c, d = classification_metrics(ytest, y_DGTDA)
    acc_DGTDA.append(a)
    prec_DGTDA.append(b)
    rec_DGTDA.append(c)
    spec_DGTDA.append(d)




Display Classification result for CMDA

In [8]:
print(np.mean(acc_CMDA), np.std(acc_CMDA))
print(np.mean(prec_CMDA), np.std(prec_CMDA))
print(np.mean(rec_CMDA), np.std(rec_CMDA))
print(np.mean(spec_CMDA), np.std(spec_CMDA))

0.5666666666666667 0.15634719199411432
0.5630511463844797 0.1399303913338515
0.7111111111111111 0.1911627837120584
0.4222222222222223 0.23934065809486685


Display result for DGTDA

In [9]:
print(np.mean(acc_DGTDA), np.std(acc_DGTDA))
print(np.mean(prec_DGTDA), np.std(prec_DGTDA))
print(np.mean(rec_DGTDA), np.std(rec_DGTDA))
print(np.mean(spec_DGTDA), np.std(spec_DGTDA))

0.5444444444444445 0.10657403385139375
0.5264550264550265 0.106529369099707
0.6222222222222222 0.21998877636914813
0.4666666666666667 0.1632993161855452
